In [29]:
%load_ext autoreload
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer
from model import generate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"

In [17]:
base_model = AutoModelForCausalLM.from_pretrained(
  MODEL_NAME,
  dtype=torch.bfloat16,
  device_map='auto'
)
THOUGHT_TOKEN = "[THOUGHT]"
SOLUTION_TOKEN = "[SOLUTION]"
RETURN_TOKEN = "[RETURN]"
SPECIAL_TOKENS = [THOUGHT_TOKEN, SOLUTION_TOKEN, RETURN_TOKEN]
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens(
    {"additional_special_tokens": SPECIAL_TOKENS}
)
base_model.resize_token_embeddings(len(tokenizer))

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Embedding(151668, 896)

In [23]:
inputs = tokenizer(["The quick brown"], return_tensors="pt").to(base_model.device)
print(inputs)

{'input_ids': tensor([[  785,  3974, 13876]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}


In [ ]:
thought_token_id = tokenizer.convert_tokens_to_ids(THOUGHT_TOKEN)
solution_token_id = tokenizer.convert_tokens_to_ids(SOLUTION_TOKEN)
return_token_id = tokenizer.convert_tokens_to_ids(RETURN_TOKEN)

gen_out = base_model.generate(**inputs, 
  custom_generate=generate.generate,
  thought_token_id=thought_token_id,
  solution_token_id=solution_token_id,
  return_token_id=return_token_id,
  eos_token_id=tokenizer.eos_token,
  use_cache=False
)
print(tokenizer.batch_decode(gen_out)) 

['The quick brown fox jumps over a lazy dog, and the word "quick" is 3 letters long. How']
